# Example regression for RNA IPS (same applies for other -omics and cells)
Code from Mike A. Nalls

In [ ]:
import os
import sys
import argparse
import math
import time
import h5py
import joblib
import subprocess
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm
from scipy import stats


#! pip install --upgrade tables
import tables

In [ ]:
#! pip install statutils
import statutils
from statutils.multi_comparison import p_adjust

In [ ]:
# Set up variables
df = pd.read_csv("analysis_table_ips.csv", engine='c')
df[['Group','GBA_Group','PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8','PC9','PC10']].describe()
df.shape

In [ ]:
predictors_list = ['GBA_Group']
covs_string = 'PC1 + PC2'

In [ ]:
# Need to update column headers since some start with numbers and python doesnt like that
columns = df.columns
new_columns = ['GENE_' + column if index >= 13 else column for index, column in enumerate(columns)]
df.columns = new_columns
first_14_columns = df.iloc[:, :14]  # Selecting all rows and the first 12 columns
print(first_14_columns)


In [ ]:
df.columns = df.columns.str.replace('-', '_')
df.columns = df.columns.str.replace('.', '_')
column_headers = list(df.columns)
outcomes_list = column_headers[13:]

## Run regression

In [ ]:

results = []

for outcome in range(len(outcomes_list)):
  for predictor in range(len(predictors_list)):
    outcome_name = outcomes_list[outcome]
    predictor_name = predictors_list[predictor]
    print("Trying " + outcome_name + " ~ " + predictor_name + " wish me luck!")
    this_formula = outcomes_list[outcome] + " ~ " + "df['" + predictors_list[predictor] + "']" + " + " + covs_string
    reg_model = sm.ols(formula=this_formula, data=df)
    fitted = reg_model.fit()
    beta_coef  = fitted.params.loc["df['" + predictors_list[predictor] + "']"]
    beta_se  = fitted.bse.loc["df['" + predictors_list[predictor] + "']"]
    p_val = fitted.pvalues.loc["df['" + predictors_list[predictor] + "']"]
    print(outcome_name, predictor_name, beta_coef, beta_se, p_val)
    results.append((outcome_name, predictor_name, beta_coef, beta_se, p_val))

output = pd.DataFrame(results, columns=('OUTCOME', 'PREDICTOR', 'BETA_COEF', 'BETA_SE','P_VAL'))

## Do some multi-test and export results

In [ ]:
output['P_VAL_FDR']=p_adjust(output['P_VAL'], method="BH")

output.to_csv("./regressions/GBA1_omics_gene_Expression_vs_group_IPS.csv", index=False)

output.describe()

## Check some hits

In [ ]:
hits = output[output['P_VAL_FDR'] < 0.05]
#hits.to_csv("./regressions/results_filtered_covs_ips.csv", index=False)
hits.describe()

# Generate QQ plot for inflation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import seaborn as sns

In [ ]:
# QQ plot function
def qqplot(pval_array, title, ax):
    logp = -np.log10(pval_array)  # Log10 transformation
    logp = np.sort(logp)
    intermid = 1/logp.shape[0]
    expected_logp = -np.log10(np.linspace(intermid, 1, logp.shape[0]))  # Log10 transformation
    expected_logp = np.sort(expected_logp)
    dat = pd.DataFrame({'p': logp, 'expected_p': expected_logp}, columns=['p', 'expected_p'])
    del logp
    del expected_logp
    X_plot = [np.min(dat['expected_p']), np.max(dat['expected_p'])]  # Adjusted range
    qqplot_res = sns.scatterplot(data=dat, x="expected_p", y="p", ax=ax)
    ax.plot(X_plot, X_plot, color='r')
    ax.set_title(title)
    ax.set_xlabel('Expected -log10(p-values)')  # Adjusted label
    ax.set_ylabel('Observed -log10(p-values)')  # Adjusted label
    
    # Calculate lambda
    observed_median = np.median(dat['p'])
    expected_median = np.median(dat['expected_p'])
    lambda_val = observed_median / expected_median
    
    # Annotate the plot with lambda value
    ax.text(0.05, 0.95, f'$\lambda$ = {lambda_val:.2f}', transform=ax.transAxes, fontsize=12,
            verticalalignment='top')

# Load data for the PCs 1-10 condition
file_path = './regressions/GBA1_omics_gene_Expression_vs_group_IPS.csv'
column_name = 'P_VAL'
title = 'Q-Q Plot IPS RNA P-Value PCs 1-2'

# Read data
data = pd.read_csv(file_path)
observed_pvalues = data[column_name]

# Create a single subplot
fig, ax = plt.subplots(1, 1, figsize=(8, 6))

# Generate QQ plot
qqplot(observed_pvalues, title, ax=ax)

# Adjust layout
plt.tight_layout()

# Save the plot (optional)
plt.savefig('./regressions/QC/ips_qq_plot_2PCs_log10.png', format='png')

# Display the plot
plt.show()